In [1]:
import genvarloader as gvl
import polars as pl
from pathlib import Path

/iblm/netapp/home/dlaub/miniforge3/envs/dna-lm/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
data_dir = Path('/iblm/netapp/data4/shared_dir/hyena_dna_collab/downstream_tasks/iPSCORE_cvpc')

In [3]:
with open(data_dir / 'samples_gt.txt') as f:
    samples = f.read().splitlines()
samples = pl.DataFrame({'sample': samples})

In [7]:
files = (
    pl.read_csv(data_dir / 'coverage_bigwigs.txt', has_header=False, new_columns=['path'])
    .with_columns(
        sample=pl.col('path').str.split('/').list.get(1)
    )
    .with_columns(
        path=str(data_dir / 'final_genotypes') + '/' + pl.col('sample') + '/' + 'coverage_' + pl.col('sample') + '.bw'
    )
)
files = samples.join(files, on='sample', how='left')
files.head()

sample,path
str,str
"""001a6b85-bf54-…","""/iblm/netapp/d…"
"""001b47b2-67fc-…","""/iblm/netapp/d…"
"""011b5bae-c25d-…","""/iblm/netapp/d…"
"""01fe3fd3-5392-…","""/iblm/netapp/d…"
"""024f3a84-e1ec-…","""/iblm/netapp/d…"


In [9]:
files.write_csv(data_dir / 'sample2bw.csv')

In [5]:
bws = {s: files[s] for s in samples}

In [6]:
bw_reader = gvl.BigWigs('cov', bws)

In [7]:
ztracks = gvl.ZarrTracks.from_reader(
    bw_reader,
    '/iblm/netapp/data4/shared_dir/hyena_dna_collab/downstream_tasks/iPSCORE_cvpc/rna_coverage.zarr',
)

  0%|          | 0/6370 [00:00<?, ?it/s]

KeyboardInterrupt: 